لنعمل الآن مع تضمين الكلمات. لقد كنا نركز غالبا على النصوص، لكننا الآن سننظر في الكلمات نظرة مباشرة باستعمال ووردتوفيك word2vec. وسنرى كيف تكون هذه التضمينات، ثم سنستعملها لاستكشاف تشابه الكلمات.


فلنبدأ بتجهيز بياناتنا.


In [1]:
from text_analytics import TextAnalytics
import os
import pandas as pd

ai = TextAnalytics()
ai.data_dir = os.path.join("..", "data")
print("Done!")

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


Done!


والآن، يستغرق تعلم تضمين الكلمات وقتا طويلا جدا أطول مما تستغرقه المهام التي كنا نجريها. ولذلك سنرفع تضمينات كلمات مدربة مسبقا. فإذا *كنت* تريد أن تجري ذلك بنفسك فالكود البرمجي هنا مضافا إليه علامة التعليق #. ويمكن أن تجد تفاصيل أكثر في حزمة * text_analytics*.

In [2]:
file = "economic.nyt.1931-2016.gz"
file = os.path.join(ai.data_dir, file)
#df = pd.read_csv(file)
#print(df)

#ai.train_word2vec(df)

ai.word_vectors = ai.deserialize("w2v_embedding", file + ".w2v_embedding.json")
ai.word_vectors_vocab = ai.deserialize("w2v_vocab", file + ".w2v_vocab.json")
    
print(ai.word_vectors)
print(list(ai.word_vectors_vocab.keys())[0:20])

[[-0.1235968   0.04748945  0.16285081 ...  0.03241363 -0.03300684
   0.26601192]
 [-0.0030584   0.04572405  0.10358463 ... -0.00173236 -0.10827369
   0.20938973]
 [-0.19914334  0.03242941  0.0766376  ...  0.12763536  0.02289162
   0.0425519 ]
 ...
 [ 0.05791736  0.24080583  0.11852352 ...  0.13066971  0.03170295
   0.20842037]
 [-0.05558461  0.00972249  0.02807915 ...  0.00696762  0.19038115
   0.08795328]
 [ 0.04808212  0.17134945 -0.06976075 ...  0.02019391  0.03982212
   0.0915439 ]]
['the_DET', 'of_ADP', 'a_DET', 'and_CCONJ', 'in_ADP', 'number_NOUN', 'to_PART', 'to_ADP', 'for_ADP', 'on_ADP', 'at_ADP', 'is_AUX', 'by_ADP', 'was_AUX', 'with_ADP', 'that_SCONJ', 'as_SCONJ', 'his_DET', 'from_ADP', 'it_PRON']


لقد رفعنا الآن تضمينات الكلمات. وقد دربت تلك التضمينات على المقالات الافتتاحية في *صحيفة نيويورك تايمز* من عام 1931 إلى عام 2016. ولنجهزهم للعمل الآن.

In [3]:
#Build an index of each word
y = list(ai.word_vectors_vocab.keys())

لنلق نظرة على بعض كلماتنا.

In [4]:
import random

sample = random.sample(ai.word_vectors_vocab.keys(), 10)
print(sample)

['shovels_VERB', 'resplendent_VERB', 'manguel_PROPN', 'chappell_PROPN', 'northwoods_NOUN', 'warble_ADJ', 'ryberg_PROPN', 'advances_PROPN', 'paradorn_PROPN', 'courtin_PROPN']


من المؤكد أنك ستلاحظ فورا أن هذه ليست كلمات عادية وحسب! وإنما أسند إليها وسوم من نحو: اسم، وفعل. فما الذي حدث؟
أولا، استعملنا PMI للوقوف على العبارات *قبل* تدريبنا لتضمينات الكلمات. فمثلا نظرنا إلى "downton abbey" في العينة العشوائية التي نبحث عنها. وهي كيان واحد، واستعمالنا PMI هنا يوضح كيف أن دلالة الكلمتين مجتمعة تختلف عن دلالتهما منفصلتين.
ثانيا، نرى وسوما، مثل: اسم، وفعل ... . وتلك تعرف بوسوم أقسام الكلام أو POS اختصارا. وتشير إلى القسم النحوي الذي تنتمي إليه كل كلمة. وقد اعتمدنا في ذلك على حزمة * spaCy*. ويمكن أن ترى آلية وكيفية ذلك بالنظر في الدالتين *train_word2vec* و* clean* في حزمتنا *text_analytics*. وسيتيح لنا ذلك التصنيفات أدناه التي قد تكون معروفة بالنسبة لك من قبل. وإن لم يكن كذلك، استعن بهذا الرابط لقراءة المزيد عنها [here](https://universaldependencies.org/u/pos/all.html).


| Open-class words  |  Closed-class words  |  Other  |
| :------------- | :----------:            | ------: |
|ADJ (adjective)         | ADP (preposition) 	                   | PUNCT (punctuation)  |
|ADV (adverb)	         | AUX (auxiliary verb)	                   | SYM (symbol)    |
|INTJ (interjection)         | CCONJ (coordinating conjunction)                   | X  (misc)     |
|NOUN (noun) 	         | DET (determiner, like "the")	                   |         |
|PROPN (proper noun)      | NUM (number)	                   |         |
|VERB (verb)	         | PART  (participle)                  |         |
                 | PRON (pronoun)                   |         |
	             | SCONJ (subordinating conjunction)                   |	     |

هنا توضيح بسيط، وهو أن *open-class words* عبارة عن تصنيفات مفتوحة غير محدودة الكلمات في كل لغة، حيث يمكننا مثلا توليد أسماء جديدة دائما. ولذلك تلك الوسوم هي ما سنعلم إسنادها للكلمات التي لم تُر من قبل. أما *closed-class words* فهي تصنيفات محدودة وثابتة. وكل كلمات هذا النوع في الإنجليزية مثلا عبارة عن كلمات وظيفية. إذن لدينا بعض المعلومات المتعلقة بكلمات هذا النوع، وسنكون دائما على معرفة بها.

إذن لماذا أضفنا هذه الوسوم النحوية قبل أن ندرب على تضمين الكلمات؟ تلك هي أسهل الأساليب لإزالة الغموض عن الكلمات. ولقد تكلمنا في هذا المبحث عن كلمات مثل "table" التي قد تكون اسما كما قد تكون فعلا. وذلك النوع من الكلمات قد يكون في تغير فسمه الكلامي بمعان مختلفة تماما وأحيانا غير متصلّة أبدا. ولذلك نضيف القسم الكلامي للكلمات حتى نتتبع معنى الكلمة الذي نهدف للتعامل معه.

والآن لنبدأ العمل مع الأكواد البرمجية! سيقف الكود البرمجي أدناه على كلمة واحدة وقوفا عشوائيا، ثم يظهرها لنا، وبعد ذلك يكشف عن الكلمات الخمس الأكثر شبها بها. ويحسب *التشابه* هنا باستعمال مقياس تشابه جيب التمام بالتطبيق على الكلمات بدلا من النصوص.


In [5]:
sample = random.sample(ai.word_vectors_vocab.keys(), 1)[0]
index = ai.word_vectors_vocab[sample]
sample, closest = ai.linguistic_distance(x = ai.word_vectors, y = y, sample = index, n = 5, metric="cosine")

print(sample, closest)

embarrassingly_ADV ['shockingly_ADV', 'curiously_ADV', 'utterly_ADV', 'unimaginative_ADJ', 'hopelessly_ADV']


هذا كل شيء! في كل مرة تعيد فيها تشغيل الكود البرمجي سترى مجموعة مختلفة من الكلمات المتشابهة.

ولنرفع تضمين الكلمات الآن من مدونة خطابات الكونقرس التي تنطوي على نفس الفترة الزمنية.


In [6]:
file2 = "economic.congress.1931-2016.gz"
file2 = os.path.join(ai.data_dir, file2)

congress_word_vectors = ai.deserialize("w2v_embedding", file2 + ".w2v_embedding.json")
congress_word_vectors_vocab = ai.deserialize("w2v_vocab", file2 + ".w2v_vocab.json")

#Build an index of each word
congress_y = list(congress_word_vectors_vocab.keys())

والآن اختر أي كلمة من قاعدة بيانات واحدة، فإذا وجدت في كلا القاعدتين، احسب الكلمات الأكثر شبها بها من المدونات 
لنقف الآن على كلمة واحدة من قاعدة بيانات واحدة، وإذا وجدت في كلا القاعدتين، نبحث عن الكلمات الأكثر شبها بها في المدونتين.


In [9]:
sample = random.sample(ai.word_vectors_vocab.keys(), 1)[0]
index = ai.word_vectors_vocab[sample]

if sample in congress_word_vectors_vocab:
    print(sample)
    congress_index = congress_word_vectors_vocab[sample]
    
    sample, closest = ai.linguistic_distance(x = ai.word_vectors, y = y, sample = index, n = 10, metric="cosine")
    print("\n", "NYT", sample, closest)
    
    sample, closest = ai.linguistic_distance(x = congress_word_vectors, y = congress_y, sample = congress_index, n = 10, metric="cosine")
    print("\n", "Congress", sample, closest)
    
else:
    print("Try again! This word is not found in both corpora.")

furs_VERB

 NYT furs_VERB ['damasks_NOUN', 'blouses_VERB', 'washandwear_NOUN', 'redingotes_NOUN', 'crushable_ADJ', 'polkadot_ADJ', 'pucci_ADJ', 'negligees_NOUN', 'tunics_PROPN', 'charmeuse_NOUN']

 Congress furs_VERB ['lipsticks_NOUN', 'toys_VERB', 'silverware_VERB', 'bicycles_VERB', 'dyes_VERB', 'medicinals_NOUN', 'lotions_NOUN', 'shampoos_NOUN', 'handbags_VERB', 'motorcycles_VERB']
